## Segmenting Clustering Neighborhoods in Toronto

---






### Import libraries:

In [0]:
import json
import folium
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from bs4 import BeautifulSoup
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.datasets.samples_generator import make_blobs

### Scrape the wiki page to extract the table and convert into Pandas dataframe:

In [0]:
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(webpage.content,'lxml')
table = soup.find_all('table')[0]
postal_code_neighbourhood_df = pd.read_html(str(table), header=0)[0]
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned:

In [0]:
postal_code_neighbourhood_df = postal_code_neighbourhood_df[(postal_code_neighbourhood_df.Borough != 'Not assigned')]
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough:

So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park:


In [0]:
postal_code_neighbourhood_df.Neighbourhood.replace("Not assigned", postal_code_neighbourhood_df.Borough, inplace=True)
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


###Group the neighbourhood by the postal code: 

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table:

In [0]:
postal_code_group_neighbourhood_df = postal_code_neighbourhood_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
postal_code_group_neighbourhood_df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Download the geospatial data into a data frame:

In [0]:
geo_data_url = "https://cocl.us/Geospatial_data"
!wget -q -O 'toronto_m.geospatial_data.csv' geo_data_url
postal_code_geo_df = pd.read_csv(geo_data_url)
postal_code_geo_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [0]:
postal_code_neighbourhood_geo_df = postal_code_neighbourhood_df.set_index('Postcode').join(postal_code_geo_df.set_index('Postal Code'))
postal_code_neighbourhood_geo_df = postal_code_neighbourhood_geo_df.reset_index()
postal_code_neighbourhood_geo_df.head(10)

,index,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [0]:
group_neighbourhood_geo_df = postal_code_group_neighbourhood_df.set_index('Postcode').join(postal_code_geo_df.set_index('Postal Code'))
group_neighbourhood_geo_df = group_neighbourhood_geo_df.reset_index()
group_neighbourhood_geo_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [0]:
print('The dataframe associated to Toronto has {} boroughs and {} neighbourhoods.'.format(
        len(postal_code_neighbourhood_df['Borough'].unique()),
        postal_code_neighbourhood_df.shape[0]
    )
)

The dataframe associated to Toronto has 11 boroughs and 211 neighbourhoods.
